In [15]:
import cv2
import dlib
import os
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image

def classify_faces(source_folder, output_folder):
    # Load the DataFrame from the JSON file
    df = pd.read_json(source_folder)
    face_encodings = {}
    for index in range(len(df)):
        # Get face encoding from the DataFrame
        vector = np.array(df['image_encode'].iloc[index], dtype=np.uint8)  # Convert to uint8
        face_encoding = df['encode'].iloc[index]  # Convert to uint8
        if  len(face_encoding):
                    # Kiểm tra xem đã có folder cho người này chưa
                    person_folder = None
                    for person, encoding in face_encodings.items():
                        # Convert face_encoding và encoding thành numpy arrays
                        face_encoding_np = np.array(face_encoding)
                        encoding_np = np.array(encoding)

                        # So sánh mã hóa để xác định xem có thuộc về cùng một người hay không
                        if cv2.norm(face_encoding_np, encoding_np, cv2.NORM_L2) < 0.4:
                            person_folder = person
                            break

                    if not person_folder:
                        # Nếu không có folder cho người này, tạo mới
                        person_folder = os.path.join(output_folder, f"Person_{len(face_encodings) + 1}")
                        os.makedirs(person_folder)
                        face_encodings[person_folder] = face_encoding

        # Assuming the shape is (100, 100, 3), adjust accordingly
        # If it's grayscale, reshape to (height, width)
                    if vector.shape[2] == 1:
                        vector = vector.squeeze()

                    saved_image_path = os.path.join(person_folder,f"{df['name'].iloc[index]}.png")  # Add the file extension
                    Image.fromarray(vector).save(saved_image_path)

if __name__ == "__main__":
    source_folder = "crop_face_encode.json"
    output_folder = "./similar_faces_encode/"

    classify_faces(source_folder, output_folder)
